In [15]:
import pandas as pd

In [4]:
test = pd.DataFrame({"Song Title": ["We found love", "Talking body", "Habits", "S&M"],
                   "Artist": ["Rihanna, Calvin Harris", "Tove Lo", "Tove Lo", "Rihanna"],
                   "Listening Time": [198, 5, 67, 89]})

test

,Song Title,Artist,Listening Time
0,We found love,"Rihanna, Calvin Harris",198
1,Talking body,Tove Lo,5
2,Habits,Tove Lo,67
3,S&M,Rihanna,89


In [3]:

def load_data_deezer():
    dee = pd.read_excel('data/deezer-data_2175171744.xlsx', sheet_name='10_listeningHistory')
    dee = dee.drop(columns=['Platform Name', 'Platform Model'])
    dee.Date = pd.to_datetime(dee.Date,format='%Y-%m-%d %H:%M:%S')
    dee = dee.rename(columns={"Listening Time" : "Listening Time (s)"})
    dee['conn_country']=dee['ISRC'].apply(lambda x: x[0:2])
    return dee


def load_data_spotify():
    spot = pd.read_json('data/Spotify_Audio_2013-2024_251124.json')
    spot = spot.iloc[:,0:9]
    spot['Listening Time (s)']=round(spot['ms_played']/1000,0)
    spot = spot.drop(columns=['platform', 'ms_played'])

    spot = spot.rename(columns={'ts': 'Date', 
                     'ip_addr':"IP Address",
                      'master_metadata_track_name': 'Song Title',
                      'master_metadata_album_artist_name' : 'Artist', # This is an approximation since Spotify doesn't take the song artist but the album artist
                      'master_metadata_album_album_name' : 'Album Title'})

    spot.Date = pd.to_datetime(spot.Date, format ="%Y-%m-%dT%H:%M:%SZ" )
    
    return spot


def load_data(dee,spot):
    data = pd.concat([dee,spot])
    data = data[data['Song Title'].isna()== False]
    data['Year'] = data['Date'].dt.year
    data = data[data.Year > 2018]
    data['spotify_track_uri'] = data['spotify_track_uri'].astype(str) 
    data['Platform']=data['spotify_track_uri'].apply(lambda x: "Deezer" if x == 'nan' else "Spotify")
    return data

In [13]:
dee = load_data_deezer()
spot = load_data_spotify()
data = load_data(dee,spot)
years =[2020, 2024]
data_year = data[data["Year"].between(years[0], years[1])]

In [17]:
data_artists = data_year
#data_artists['Artist'] = data_artists['Artist'].str.split(',')# Dans le cas où 2 artistes ont contribué à une chanson, on duplique le temps d'écoute pour avoir chacun le temps d'écoute de la chanson
data_artists = data_artists.explode('Artist')  
data_artists = data_artists.reset_index(drop=True) # Réinitialise l'index 
data_artists.head()

,Song Title,Artist,ISRC,Album Title,IP Address,Listening Time (s),Date,conn_country,spotify_track_uri,Year,Platform
0,"Les Rois Du Monde (Thème De ""Roméo Et Juliette...",Best Of Hits,FR0W61000893,Best Of Hits Vol. 74,37.164.130.31,198.0,2021-08-09 14:50:11,FR,nan,2021,Deezer
1,We Found Love,Rihanna,USUM71115507,Talk That Talk,93.12.51.200,1.0,2023-05-05 17:14:06,US,nan,2023,Deezer
2,We Found Love,Calvin Harris,USUM71115507,Talk That Talk,93.12.51.200,1.0,2023-05-05 17:14:06,US,nan,2023,Deezer
3,We Found Love,Rihanna,USUM71115507,Talk That Talk,37.170.143.159,214.0,2023-05-08 11:26:27,US,nan,2023,Deezer
4,We Found Love,Calvin Harris,USUM71115507,Talk That Talk,37.170.143.159,214.0,2023-05-08 11:26:27,US,nan,2023,Deezer


In [20]:
fav = ['Tove Lo', 'FLETCHER']
data_filtered = data_artists[data_artists["Artist"].isin(fav)]

In [ ]:

top_artists = data_artists.pivot_table(
    index="Artist", columns="Year", values="Listening Time (s)", aggfunc="sum", fill_value=0
    )
top_artists=top_artists.astype(float)
top_artists['Total']=top_artists.sum(axis=1)
top_artists = top_artists.sort_values(by="Total", ascending=False).head(10)
top_artists = round(top_artists/60)
top_artists = top_artists.drop('Total', axis=1)

In [54]:
top_songs=top_songs.sort_values(by="Total", ascending=False)
top_songs.head(10)

Year,2020,2021,2022,2023,2024,Total
Song Title,,,,,,
Angela,29030.0,3095.0,1245.0,5.0,0.0,33375.0
"Alone, Pt. II",27039.0,1378.0,665.0,358.0,35.0,29475.0
MAMACITA,22862.0,1574.0,1961.0,187.0,0.0,26584.0
Salt,21585.0,1382.0,360.0,678.0,180.0,24185.0
Like It Is,17057.0,3896.0,2158.0,182.0,0.0,23293.0
In Your Eyes (feat. Alida),16706.0,5599.0,624.0,208.0,0.0,23137.0
All Falls Down (feat. Juliander),17784.0,2371.0,1594.0,199.0,9.0,21957.0
Las vegas,15935.0,2567.0,1794.0,686.0,0.0,20982.0
Thank God,18928.0,1341.0,412.0,0.0,0.0,20681.0


In [1]:
import pandas as pd 
data = {'titre': ['Chanson 1', 'Chanson 2', 'Chanson 3'], 
        'artiste': ['Artiste A', 'Artiste B, Artiste C', 'Artiste D'], 
        'temps_écoute': [180, 240, 150]} 
df = pd.DataFrame(data) 
df['artiste'] = df['artiste'].str.split(', ')  # Sépare la chaîne par des virgules et des espaces 
df = df.explode('artiste')  # Explose la colonne 'artiste' 
df = df.reset_index(drop=True) # Réinitialise l'index 
print(df)

       titre    artiste  temps_écoute
0  Chanson 1  Artiste A           180
1  Chanson 2  Artiste B           240
2  Chanson 2  Artiste C           240
3  Chanson 3  Artiste D           150


In [40]:
for index, row in test.iterrows():
    for artist in fav :
        if artist in row["Artist"] :
            data_final.loc[len(data_final)] = row 

In [41]:
data_final

,Song Title,Artist,ISRC,Album Title,IP Address,Listening Time (s),Date,conn_country,spotify_track_uri,Year,Platform
0,We found love,"Rihanna, Calvin Harris",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Talking body,Tove Lo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Habits,Tove Lo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
fav = ['Calvin Harris', 'Tove Lo']

In [31]:
artists = test['Artist'].str.split(',').explode().str.strip().unique()

In [23]:
artists.sort()
artists

array(['(G)I-DLE', '-M-', '. Harmonize', ..., 'Øudjat', 'Üm',
       '박혜진 Park Hye Jin'], dtype=object)

In [16]:
dee = pd.read_excel('data/deezer-data_2175171744.xlsx', sheet_name='10_listeningHistory', )
dee = dee.drop(columns=['Platform Name', 'Platform Model'])
dee.Date = pd.to_datetime(dee.Date,format='%Y-%m-%d %H:%M:%S')
dee = dee.rename(columns={"Listening Time" : "Listening Time (s)"})
dee['conn_country']=dee['ISRC'].apply(lambda x: x[0:2])


In [25]:
dee.head()

,Song Title,Artist,ISRC,Album Title,IP Address,Listening Time (s),Date,conn_country
0,"Les Rois Du Monde (Thème De ""Roméo Et Juliette...",Best Of Hits,FR0W61000893,Best Of Hits Vol. 74,37.164.130.31,198,2021-08-09 14:50:11,FR
1,We Found Love,"Rihanna, Calvin Harris",USUM71115507,Talk That Talk,93.12.51.200,1,2023-05-05 17:14:06,US
2,We Found Love,"Rihanna, Calvin Harris",USUM71115507,Talk That Talk,37.170.143.159,214,2023-05-08 11:26:27,US
3,We Found Love,"Rihanna, Calvin Harris",USUM71115507,Talk That Talk,147.161.184.198,216,2023-05-16 11:48:39,US
4,Beggin,Madcon,NOBLV0703011,Beggin,84.102.140.147,218,2020-03-17 18:16:50,NO


In [17]:
spot = pd.read_json('data/Spotify_Audio_2013-2024_251124.json')
spot.head()

,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
0,2013-05-14T17:02:38Z,WebPlayer (websocket RFC6455),221999,FR,81.57.133.167,Always On The Run,Yuksek,Living On The Edge Of Time,spotify:track:3McVJ4pdwNMP4RJtsJAogG,None,None,None,autoplay,nextbtn,False,False,False,NaN,False
1,2013-05-14T17:04:37Z,Windows 7 (Unknown Ed) SP1 [x86 0],56188,FR,81.57.133.167,Can't Hold Us (feat. Ray Dalton),Macklemore & Ryan Lewis,The Heist,spotify:track:3BE0getU08uXas8ZjULABA,None,None,None,unknown,popup,False,True,False,NaN,False
2,2013-05-14T17:05:15Z,Windows 7 (Unknown Ed) SP1 [x86 0],39020,FR,81.57.133.167,Suit & Tie (feat. Jay-Z),Justin Timberlake,The 20/20 Experience (Deluxe Version),spotify:track:6vt0I1cw1YmAIKDJvHVIM5,None,None,None,popup,popup,False,True,False,NaN,False
3,2013-05-14T17:05:19Z,Windows 7 (Unknown Ed) SP1 [x86 0],3947,FR,81.57.133.167,Down The Road,C2C,Tetra,spotify:track:2cx9XbBUC76pi3ZpGk1MJ5,None,None,None,popup,popup,False,True,False,NaN,False
4,2013-05-14T17:05:45Z,Windows 7 (Unknown Ed) SP1 [x86 0],26225,FR,81.57.133.167,Hot Right Now (feat. RITA ORA),DJ Fresh,ORA,spotify:track:4eY1fWFPLztKDx9athN4hx,None,None,None,popup,popup,False,True,False,NaN,False


In [79]:
spot.ms_played.sum()/1000

np.float64(788052.623)

In [18]:

spot = spot.iloc[:,0:9]
spot['Listening Time (s)']=round(spot['ms_played']/1000,0)
spot = spot.drop(columns=['platform', 'ms_played'])

spot = spot.rename(columns={'ts': 'Date', 
                     'ip_addr':"IP Address",
                      'master_metadata_track_name': 'Song Title',
                      'master_metadata_album_artist_name' : 'Artist',
                      'master_metadata_album_album_name' : 'Album Title'})

spot.Date = pd.to_datetime(spot.Date, format ="%Y-%m-%dT%H:%M:%SZ" )


In [99]:
spot.head()

,Date,conn_country,IP Address,Song Title,Artist,Album Title,spotify_track_uri,Listening Time (s)
0,2013-05-14 17:02:38,FR,81.57.133.167,Always On The Run,Yuksek,Living On The Edge Of Time,spotify:track:3McVJ4pdwNMP4RJtsJAogG,222.0
1,2013-05-14 17:04:37,FR,81.57.133.167,Can't Hold Us (feat. Ray Dalton),Macklemore & Ryan Lewis,The Heist,spotify:track:3BE0getU08uXas8ZjULABA,56.0
2,2013-05-14 17:05:15,FR,81.57.133.167,Suit & Tie (feat. Jay-Z),Justin Timberlake,The 20/20 Experience (Deluxe Version),spotify:track:6vt0I1cw1YmAIKDJvHVIM5,39.0
3,2013-05-14 17:05:19,FR,81.57.133.167,Down The Road,C2C,Tetra,spotify:track:2cx9XbBUC76pi3ZpGk1MJ5,4.0
4,2013-05-14 17:05:45,FR,81.57.133.167,Hot Right Now (feat. RITA ORA),DJ Fresh,ORA,spotify:track:4eY1fWFPLztKDx9athN4hx,26.0


In [19]:
data = pd.concat([dee,spot])
data['Year'] = data['Date'].dt.year
data = data[data.Year > 2018]
data['spotify_track_uri'] = data['spotify_track_uri'].astype(str) 
data['Platform']=data['spotify_track_uri'].apply(lambda x: "Deezer" if x == 'nan' else "Spotify")
data = data[data['Song Title'].isna()== False]
data.head()

,Song Title,Artist,ISRC,Album Title,IP Address,Listening Time (s),Date,conn_country,spotify_track_uri,Year,Platform
0,"Les Rois Du Monde (Thème De ""Roméo Et Juliette...",Best Of Hits,FR0W61000893,Best Of Hits Vol. 74,37.164.130.31,198.0,2021-08-09 14:50:11,FR,nan,2021,Deezer
1,We Found Love,"Rihanna, Calvin Harris",USUM71115507,Talk That Talk,93.12.51.200,1.0,2023-05-05 17:14:06,US,nan,2023,Deezer
2,We Found Love,"Rihanna, Calvin Harris",USUM71115507,Talk That Talk,37.170.143.159,214.0,2023-05-08 11:26:27,US,nan,2023,Deezer
3,We Found Love,"Rihanna, Calvin Harris",USUM71115507,Talk That Talk,147.161.184.198,216.0,2023-05-16 11:48:39,US,nan,2023,Deezer
4,Beggin,Madcon,NOBLV0703011,Beggin,84.102.140.147,218.0,2020-03-17 18:16:50,NO,nan,2020,Deezer


In [93]:
data['Listening Time (min)']=round(data['Listening Time (s)']/60,0)
data.head()

,Song Title,Artist,ISRC,Album Title,IP Address,Listening Time (s),Date,conn_country,Album Artist,spotify_track_uri,Year,Platform,Listening Time (min)
0,"Les Rois Du Monde (Thème De ""Roméo Et Juliette...",Best Of Hits,FR0W61000893,Best Of Hits Vol. 74,37.164.130.31,198.0,2021-08-09 14:50:11,FR,NaN,nan,2021,Deezer,3.0
1,We Found Love,"Rihanna, Calvin Harris",USUM71115507,Talk That Talk,93.12.51.200,1.0,2023-05-05 17:14:06,US,NaN,nan,2023,Deezer,0.0
2,We Found Love,"Rihanna, Calvin Harris",USUM71115507,Talk That Talk,37.170.143.159,214.0,2023-05-08 11:26:27,US,NaN,nan,2023,Deezer,4.0
3,We Found Love,"Rihanna, Calvin Harris",USUM71115507,Talk That Talk,147.161.184.198,216.0,2023-05-16 11:48:39,US,NaN,nan,2023,Deezer,4.0
4,Beggin,Madcon,NOBLV0703011,Beggin,84.102.140.147,218.0,2020-03-17 18:16:50,NO,NaN,nan,2020,Deezer,4.0


In [94]:
data_platform = data[data["Year"].between(2020, 2024)]

data_platform_reshaped = data_platform.pivot_table(
    index="Year", columns="Platform", values="Listening Time (min)", aggfunc="sum", fill_value=0)
data_platform_reshaped = data_platform_reshaped.sort_values(by="Year", ascending=False)


In [95]:
data_platform_reshaped

Platform,Deezer,Spotify
Year,,
2024,846.0,5150.0
2023,4824.0,0.0
2022,7169.0,0.0
2021,8815.0,0.0
2020,37752.0,0.0
